In [1]:
from pymilvus import MilvusClient
import numpy as np
import os
import openai
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
import numpy as np
import hashlib
import time
from dotenv import load_dotenv
import os
from openai import AzureOpenAI

In [2]:
client_db = MilvusClient("./milvus_large3.db")

In [3]:
load_dotenv()

True

In [4]:
SYSTEM_PROMPT = """You are an instruct fine tuning assistant that generates data for supply chain management. Your task is to create answers to a set of questions that will be used to teach undergraduate, graduate, and executive students the principles of logistics, operations, and supply chain management.
Ensure answers are comprehensive yet precise, concise, and grounded in pre-trained knowledge and industry best practices. Tailor responses to be clear, actionable, and contextually relevant to the subject matter, avoiding unnecessary jargon. Focus on accuracy and clarity while maintaining a balance between depth and brevity to support diverse learning needs.

Guidelines:
- Provide comprehensive responses that teach the concepts clearly and practically
- Make explanations suitable for business professionals (like supply chain analyst or manager)
- Base all answers on the provided text context
- Just give the answer, no additional text or formatting
- Answer in approximately 180 to 230 words, be concise but complete and avoid unnecessary elaboration or repetition."""

In [5]:
client_embed = AzureOpenAI(
  api_key = os.getenv("AZURE_embed3lrg_API_KEY"),  
  api_version = "2024-10-21",
  azure_endpoint = os.getenv("AZURE_embed3lrg_API_BASE") 
)

In [6]:
from openai import OpenAI
 
client_kimi = OpenAI(
    api_key = os.getenv("MOONSHOT_API_KEY"),
    base_url = os.getenv("MOONSHOT_API_BASE"),
)

In [7]:
def context_func(ques):
    response = client_embed.embeddings.create(
                        input = q,
                        model= "text-embedding-3-large"
                    )
    search_res = client_db.search(
        collection_name="all_supply_chain_books",
        data=[
            response.data[0].embedding
        ],  
        limit=5,  
        search_params={"metric_type": "COSINE", "params": {}},  
        output_fields=["text"],  
    )
    retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]
    # print(json.dumps(retrieved_lines_with_distances, indent=4))
    context = "\n".join([line_with_distance[0] for line_with_distance in retrieved_lines_with_distances])
    return context

In [ ]:
import json
# start_index = 0   # Question 10 (0-based index)
# end_index = 4    # Up to but not including index 12 (Question 12)

with open("generated_questions.jsonl", "r") as f, open("output.jsonl", "w") as outfile:
    for i, line in tqdm(enumerate(f)):
        # if i >= start_index and i < end_index:
        data = json.loads(line)
        # questions.append(data['question'])
        q = data['question']
        # print(q)
        context = context_func(q)
        USER_PROMPT = f"""
        Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
        <context>
        {context}
        </context>
        <question>
        {q}
        </question>
        """
        response = client_kimi.chat.completions.create(
            model="kimi-k2-0711-preview",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": USER_PROMPT},
            ],
            temperature = 0.4,
        )
        answer = response.choices[0].message.content.strip()
        # print(answer)
        # print("="*150)
        data['answer'] = answer
        outfile.write(json.dumps(data, ensure_ascii=False) + "\n")
                
        # elif i >= end_index:
        #     break

In [ ]:
# for q in questions:
#     print(q)
#     context = context_func(q)
#     USER_PROMPT = f"""
#     Use the following pieces of information enclosed in <context> tags to provide an answer to the question enclosed in <question> tags.
#     <context>
#     {context}
#     </context>
#     <question>
#     {q}
#     </question>
#     """
#     response = client_kimi.chat.completions.create(
#         model="kimi-k2-0711-preview",
#         messages=[
#             {"role": "system", "content": SYSTEM_PROMPT},
#             {"role": "user", "content": USER_PROMPT},
#         ],
#         temperature = 0.4,
#     )
#     answer = response.choices[0].message.content.strip()
#     print(answer)
#     print("="*150)